In [1]:
import mediapipe as mp
import cv2
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Basic Detection

In [3]:
# Getting video feed
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        # Rendering
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2),
                                  )

        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.95089483
y: 0.88351
z: -0.09175369
visibility: 0.011386959
, x: 0.9443685
y: 0.8341157
z: -0.09503766
visibility: 0.0040780935
, x: 0.9501945
y: 0.8255271
z: -0.09569636
visibility: 0.0042380583
, x: 0.9569026
y: 0.8179845
z: -0.095748745
visibility: 0.0063235476
, x: 0.9276056
y: 0.85333365
z: -0.083072536
visibility: 0.0061050365
, x: 0.9222433
y: 0.85887945
z: -0.08430828
visibility: 0.00914803
, x: 0.91632414
y: 0.864261
z: -0.08523347
visibility: 0.0154836085
, x: 0.9748386
y: 0.8010755
z: 0.005733869
visibility: 0.018249044
, x: 0.91757953
y: 0.8621734
z: 0.06506542
visibility: 0.017043583
, x: 0.9779469
y: 0.8844872
z: -0.033319805
visibility: 0.032783337
, x: 0.95514077
y: 0.910717
z: -0.023588616
visibility: 0.0325941
, x: 1.1416585
y: 0.8644052
z: 0.078822725
visibility: 0.8228488
, x: 0.9239826
y: 1.0626079
z: 0.14528753
visibility: 0.83208036
, x: 1.2599797
y: 1.0794562
z: 0.034014378
visibility: 0.04334341
, x: 0.9971477
y: 1.3139193
z: 0.105789386
visibility: 0.0581

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [4]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.7135384
y: 0.7499835
z: -0.12970687
visibility: 0.9943653

In [5]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.9026979
y: 1.000645
z: -0.3357154
visibility: 0.4439971

# 3. Calculate Angles

In [6]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [6]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]


In [7]:
shoulder, elbow, wrist

([0.8881043195724487, 0.8783743381500244],
 [0.9788001179695129, 1.3374955654144287],
 [0.8884776830673218, 1.275389313697815])

In [8]:
calculate_angle(shoulder, elbow, wrist)

44.31289053689085

In [ ]:
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # or any width you prefer
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # or any height you prefer
# stage = ""
# counter = 0
# ## Setup mediapipe instance
# with mp_pose.Pose(min_detection_confidence=0.5, 
#                   min_tracking_confidence=0.5,
#                   model_complexity=2,
#                   static_image_mode=False,
#                   smooth_landmarks=False,
#                           ) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False
      
#         # Make detection
#         results = pose.process(image)
    
#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#         frame = cv2.flip(frame, 1)
        
#         # Extract landmarks
#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark
#             for i, landmark in enumerate(landmarks):
#                 print(f"Landmark {i}: x={landmark.x}, y={landmark.y}, z={landmark.z}")

#             # Get coordinates
#             angles_to_calculate = {
#                 "right_elbow_right_shoulder_right_hip": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
#                 ],
#                 "left_elbow_left_shoulder_left_hip": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
#                 ],
#                 "right_knee_mid_hip_left_knee": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
#                     [(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) / 2,
#                      (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y) / 2],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
#                 ],
#                 "right_hip_right_knee_right_ankle": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
#                 ],
#                 "left_hip_left_knee_left_ankle": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
#                 ],
#                 "right_wrist_right_elbow_right_shoulder": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
#                 ],
#                 "left_wrist_left_elbow_left_shoulder": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
#                 ],
#             }

#             # Calculate and visualize angles
#             angles = []
#             for i, (angle_name, points) in enumerate(angles_to_calculate.items()):
#                 angle = calculate_angle(points[0], points[1], points[2])
#                 angles.append(angle)
#                 cv2.putText(image, f'{angle_name}: {int(angle)}', 
#                             (50, 100 + i * 30), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

#             if angles[3] > 160 and angles[4] > 160:
#                 stage = "up"
#             if angles[3] < 90 and angles[4] < 90 and stage == "up":
#                 stage = "down"
#                 counter += 1
#         else:
#             print("No landmarks detected in this frame.")

            
#         # Render curl counter
#         # Setup status box
#         cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
#         # Rep data
#         cv2.putText(image, 'REPS', (15,12), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
#         cv2.putText(image, str(counter), 
#                     (10,60), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
#         # Stage data
#         cv2.putText(image, 'STAGE', (65,12), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
#         cv2.putText(image, stage, 
#                     (60,60), 
#                     cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
#         # Render detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
#                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
#                                  )               
        
#         # cv2.imshow('Mediapipe Feed', cv2.flip(image,1))
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

Landmark 0: x=0.5752944350242615, y=0.3633301258087158, z=-0.6873043179512024
Landmark 1: x=0.5665771961212158, y=0.2969125509262085, z=-0.6435420513153076
Landmark 2: x=0.5714081525802612, y=0.2959836721420288, z=-0.6441420912742615
Landmark 3: x=0.5770502090454102, y=0.29436957836151123, z=-0.6441099643707275
Landmark 4: x=0.5394561886787415, y=0.30071401596069336, z=-0.6795510649681091
Landmark 5: x=0.5284700393676758, y=0.3046315908432007, z=-0.6803734302520752
Landmark 6: x=0.5107538104057312, y=0.31224846839904785, z=-0.6808679699897766
Landmark 7: x=0.5512123703956604, y=0.33782315254211426, z=-0.37087395787239075
Landmark 8: x=0.46156319975852966, y=0.36662757396698, z=-0.5311490297317505
Landmark 9: x=0.5863280892372131, y=0.4316598176956177, z=-0.5715799927711487
Landmark 10: x=0.5527874827384949, y=0.4364367723464966, z=-0.6234316229820251
Landmark 11: x=0.673921525478363, y=0.6744873523712158, z=-0.21589155495166779
Landmark 12: x=0.3440682888031006, y=0.713951051235199, z=

In [ ]:
# import mediapipe as mp
# import cv2
# import numpy as np
# import torch
# import nbimporter  # Import nbimporter

# from data import NeuralNetwork  
#  # Import your model class

# model = NeuralNetwork()  # Initialize the model
# model.load_state_dict(torch.load(r'D:\PROGRAMMING\BE proj24\data.pth'))  # Load the state dict
# model.eval() 

# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Get video feed
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # or any width you prefer
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) 
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         # Make detection
#         results = pose.process(image)

#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         # Extract landmarks
#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark

#             # Prepare input data for your model
#             input_tensor = torch.tensor(angles, dtype=torch.float32).unsqueeze(0)

#             # Predict next movement
#             with torch.no_grad():
#                 predicted_angles = model(input_tensor)
#                 predicted_angles = predicted_angles.squeeze().tolist()

#             # Display predicted movement
#             for i, angle in enumerate(predicted_angles):
#                 cv2.putText(image, f'Next Angle {i+1}: {int(angle)}', 
#                             (50, 300 + i * 30),  # Adjust y-position to fit all angles
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA) 


#         # Render pose landmarks
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()


In [8]:
import mediapipe as mp
import cv2
import numpy as np
import torch
import nbimporter  # Import nbimporter

from data import NeuralNetwork  # Import your model class

# Initialize the model
model = NeuralNetwork()
model.load_state_dict(torch.load(r'D:\PROGRAMMING\BE proj24\data.pth'))
model.eval()

# Set up MediaPipe drawing and pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Open video feed
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
stage = ""
counter = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2, smooth_landmarks=False) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make pose detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks and calculate angles
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Define angle calculations (you can add other angles here)
            angles_to_calculate = {
                "right_elbow_right_shoulder_right_hip": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                ],
                "left_elbow_left_shoulder_left_hip": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                ],
                "right_knee_mid_hip_left_knee": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) / 2,
                     (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y) / 2],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                ],
                "right_hip_right_knee_right_ankle": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
                ],
                "left_hip_left_knee_left_ankle": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
                ],
                "right_wrist_right_elbow_right_shoulder": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                ],
                "left_wrist_left_elbow_left_shoulder": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                ],
            }

            # Calculate angles and store them in the list for model input
            angles = []
            for i, (angle_name, points) in enumerate(angles_to_calculate.items()):
                angle = calculate_angle(points[0], points[1], points[2])
                angles.append(angle)
                cv2.putText(image, f'{angle_name}: {int(angle)}', (50, 100 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            # Prepare input data for the model (use angles as input tensor)
            input_tensor = torch.tensor(angles, dtype=torch.float32).unsqueeze(0)

            # Predict next movement
            with torch.no_grad():
                predicted_angles = model(input_tensor).squeeze().tolist()

            # Display predicted next movement angles
            for i, angle in enumerate(predicted_angles):
                cv2.putText(image, f'Next Angle {i+1}: {int(angle)}', (50, 300 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

            # Rep counting logic
            if angles[3] > 160 and angles[4] > 160:
                stage = "up"
            if angles[3] < 90 and angles[4] < 90 and stage == "up":
                stage = "down"
                counter += 1

        # Render rep counter and stage data
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        # Render pose landmarks
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Display image
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Holistic

In [21]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [17]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh_connections  # Import face_mesh for FACE_CONNECTIONS

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5,
                          static_image_mode=False,
                          smooth_landmarks=True,
                          model_complexity=2
                            ) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,  # Use FACE_CONNECTIONS from face_mesh module
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
                        
        cv2.imshow('Holistic Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [23]:
# SPINE ANGLE

In [24]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First point  
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for back straightness check
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle for back straightness
            spine_angle = calculate_angle(shoulder, hip, knee)
            
            # Visualize spine angle
            cv2.putText(image, f'Spine Angle: {int(spine_angle)}', 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Get foot positions for feet check
            left_heel_z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            right_heel_z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z
            
            # Check if feet are lifting off the ground
            if left_heel_z > 0.25 or right_heel_z > 0.25:  # Adjust threshold as needed
                cv2.putText(image, "Feet not planted!", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Feet planted", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow('Squat Form Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Holistic

In [25]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [26]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh_connections  # Import face_mesh for FACE_CONNECTIONS

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5,
                          static_image_mode=False,
                          smooth_landmarks=True,
                          model_complexity=2
                            ) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,  # Use FACE_CONNECTIONS from face_mesh module
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
                        
        cv2.imshow('Holistic Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [27]:
# SPINE ANGLE

In [28]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First point  
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for back straightness check
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle for back straightness
            spine_angle = calculate_angle(shoulder, hip, knee)
            
            # Visualize spine angle
            cv2.putText(image, f'Spine Angle: {int(spine_angle)}', 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Get foot positions for feet check
            left_heel_z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            right_heel_z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z
            
            # Check if feet are lifting off the ground
            if left_heel_z > 0.25 or right_heel_z > 0.25:  # Adjust threshold as needed
                cv2.putText(image, "Feet not planted!", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Feet planted", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow('Squat Form Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
